# 异步爬虫

## 什么是爬虫 ?

就是客户端向服务器发请求，【批量】获取响应数据

## 异步爬虫的方式

- 多线程，多进程【不建议】:

    好处: 可以为相关阻塞的操作单独开启线程或进程，阻塞操作就可以异步执行
    坏处: 无法无限制的开启多进程或多线程
    
- 进程池、线程池【适当使用】:

    好处: 可以降低系统对进程或者线程创建和销毁的一个频率，从而降低系统的开销
    好处: 池中线程或进程的数量有上限【阻塞操作的数量远远高于池中进程线程的数量时，效率不明显】

In [1]:
# package
import time
# 导入线程池模块对应的类
from multiprocessing.dummy import Pool
import requests
from lxml import etree

import asyncio
import nest_asyncio
nest_asyncio.apply()

# 模拟线程池

In [ ]:
# 单线程方式执行
def getPage(str):
    print('正在下载:', str)
    time.sleep(2)
    print('下载完成', str)

nameList = ['a', 'b', 'c', 'd']
startTime = time.time()
for i in range(len(nameList)):
    getPage(nameList[i])
endTime = time.time()
print('%d second'% (endTime - startTime))

In [ ]:
# 线程池模拟
startTime = time.time()
pool = Pool(4)
# 将列表中每一列元素传递给 getPage 进行处理
pool.map(getPage, nameList)
endTime = time.time()
print(endTime - startTime)


# 利用线程池爬取 梨视频的视频数据


In [ ]:
url = 'https://www.pearvideo.com'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

# params = {
#     'start': '0',
#     'k': '迪丽热巴'
# }

In [ ]:
page_text = requests.get(url = url, headers = headers).text
tree = etree.HTML(page_text)
src = tree.xpath('//ul[@class="vervideo-blist clearfix"]//a/@href')[2]

video_src = url + '/' + src


# 携程对象

In [2]:
async def request(url):
    print('正在请求的 url: ', url)
    print('请求成功', url)
    return url
    
# 用 async 修饰的函数，调用之后返回的是一个携程对象
c = request('www.baidu.com')

# 事件循环对象

In [ ]:
# 创建一个事件循环对象
# loop = asyncio.get_event_loop()

# 将携程对象注册到事件循环中，然后启动 loop
# loop.run_until_complete(c)

# task 对象

- task 对象是基于 loop 的
- future 是基于 asyncio 的

In [ ]:
# 创建 task 对象【task 对象可以保存携程对象的状态】
loop = asyncio.get_event_loop()
# 基于 loop 创建 task 对象
task = loop.create_task(c)
print(task)
loop.run_until_complete(task)
print(task)

# future 对象

In [ ]:
# future 的使用
loop = asyncio.get_event_loop()
task = asyncio.ensure_future(c)
print(task)
loop.run_until_complete(task)
# print(task)

# 绑定回调

【为什么不执行 ?】

In [3]:
def callback_func(task):
    print('🐱🐱🐱', task.result())
    x = 200
    # pass

x = 1
loop = asyncio.get_event_loop()
# task = asyncio.ensure_future(c)
task = loop.create_task(c)
# 将回调函数绑定到任务对象中
print('---')
task.add_done_callback(callback_func)
print('+++')
loop.run_until_complete(task)

---
+++
正在请求的 url:  www.baidu.com
请求成功 www.baidu.com


'www.baidu.com'